In [26]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batchsize=128
# Training dataset
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='./mnist/', train=False, download=True,transform=transforms.ToTensor()),batch_size=batchsize)
# Test dataset
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='./mnist/', train=False, download=False,transform=transforms.ToTensor()),batch_size=batchsize)


### 按论文中P6的TABLE 1 构建

In [56]:
class MNIST_Model(nn.Module):
    def __init__(self):
        super(MNIST_Model, self).__init__()  
        self.conv1 = nn.Conv2d(1, 32, 3,)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.conv4 = nn.Conv2d(64, 64, 3)
        self.max_pooling1=torch.nn.MaxPool2d(2,2)
        self.max_pooling2=torch.nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(4*4*64, 200) 
        self.fc2 = nn.Linear(200, 10) # 10分类
 
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.max_pooling1(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.max_pooling2(x)
        
        x = x.permute((0, 2, 3, 1))
        x = x.contiguous().view(-1, 4 * 4 * 64)
        x = x.view(-1, 4 * 4 * 64)
        
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5)
        x = self.fc2(x)
        x = F.relu(x)
#         x = F.log_softmax(x,dim=1)
        return x

model = MNIST_Model().cuda()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9,nesterov=True)

In [57]:
criteria = nn.CrossEntropyLoss()
def train(epoch):
    train_loss = 0
    train_correct = 0
    bs=10
    model.train()
    for batch_index, (data, target) in enumerate(train_loader):
        data, target = Variable(data).cuda(), Variable(target).cuda()
        optimizer.zero_grad()
        output = model(data)
#         loss = F.nll_loss(output, target)
        loss = criteria(output, target.long())
        loss.backward()
        optimizer.step()
        #记录当前训练集的结果
        pred = output.argmax(dim=1, keepdim=True)
        train_correct += pred.eq(target.long().view_as(pred)).sum().item()
        # 显示当前轮的
        if batch_index % bs == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}'.format(
                epoch, batch_index, len(train_loader),
                100.0 * batch_index / len(train_loader), loss.item()/ bs))
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        train_loss/len(train_loader.dataset), train_correct, len(train_loader.dataset),
        100.0 * train_correct / len(train_loader.dataset)))
    
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()
            output = model(data)
            test_loss += criteria(output, target.long()).item()
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [58]:
total_epoch = 50
for epoch in range(1, total_epoch+1):
    train(epoch)
    test()

Train Epoch: 1 [0/79 (0%)]	Train Loss: 0.230261
Train Epoch: 1 [10/79 (13%)]	Train Loss: 0.230338
Train Epoch: 1 [20/79 (25%)]	Train Loss: 0.229903
Train Epoch: 1 [30/79 (38%)]	Train Loss: 0.226572
Train Epoch: 1 [40/79 (51%)]	Train Loss: 0.185323
Train Epoch: 1 [50/79 (63%)]	Train Loss: 0.097797
Train Epoch: 1 [60/79 (76%)]	Train Loss: 0.037877
Train Epoch: 1 [70/79 (89%)]	Train Loss: 0.151091

Train set: Average loss: 0.0000, Accuracy: 4432/10000 (44%)


Test set: Average loss: 0.0041, Accuracy: 8573/10000 (86%)

Train Epoch: 2 [0/79 (0%)]	Train Loss: 0.051102
Train Epoch: 2 [10/79 (13%)]	Train Loss: 0.044241
Train Epoch: 2 [20/79 (25%)]	Train Loss: 0.022856
Train Epoch: 2 [30/79 (38%)]	Train Loss: 0.032169
Train Epoch: 2 [40/79 (51%)]	Train Loss: 0.009698
Train Epoch: 2 [50/79 (63%)]	Train Loss: 0.016683
Train Epoch: 2 [60/79 (76%)]	Train Loss: 0.002126
Train Epoch: 2 [70/79 (89%)]	Train Loss: 0.075123

Train set: Average loss: 0.0000, Accuracy: 9213/10000 (92%)


Test set: Average 

Train Epoch: 16 [70/79 (89%)]	Train Loss: 0.000239

Train set: Average loss: 0.0000, Accuracy: 9945/10000 (99%)


Test set: Average loss: 0.0002, Accuracy: 9926/10000 (99%)

Train Epoch: 17 [0/79 (0%)]	Train Loss: 0.000011
Train Epoch: 17 [10/79 (13%)]	Train Loss: 0.000824
Train Epoch: 17 [20/79 (25%)]	Train Loss: 0.002423
Train Epoch: 17 [30/79 (38%)]	Train Loss: 0.003810
Train Epoch: 17 [40/79 (51%)]	Train Loss: 0.000496
Train Epoch: 17 [50/79 (63%)]	Train Loss: 0.000488
Train Epoch: 17 [60/79 (76%)]	Train Loss: 0.000678
Train Epoch: 17 [70/79 (89%)]	Train Loss: 0.001326

Train set: Average loss: 0.0000, Accuracy: 9923/10000 (99%)


Test set: Average loss: 0.0002, Accuracy: 9938/10000 (99%)

Train Epoch: 18 [0/79 (0%)]	Train Loss: 0.000207
Train Epoch: 18 [10/79 (13%)]	Train Loss: 0.013354
Train Epoch: 18 [20/79 (25%)]	Train Loss: 0.005437
Train Epoch: 18 [30/79 (38%)]	Train Loss: 0.001375
Train Epoch: 18 [40/79 (51%)]	Train Loss: 0.000165
Train Epoch: 18 [50/79 (63%)]	Train Loss: 0.

Train Epoch: 32 [20/79 (25%)]	Train Loss: 0.002225
Train Epoch: 32 [30/79 (38%)]	Train Loss: 0.000355
Train Epoch: 32 [40/79 (51%)]	Train Loss: 0.004229
Train Epoch: 32 [50/79 (63%)]	Train Loss: 0.000079
Train Epoch: 32 [60/79 (76%)]	Train Loss: 0.000000
Train Epoch: 32 [70/79 (89%)]	Train Loss: 0.000072

Train set: Average loss: 0.0000, Accuracy: 9961/10000 (100%)


Test set: Average loss: 0.0002, Accuracy: 9928/10000 (99%)

Train Epoch: 33 [0/79 (0%)]	Train Loss: 0.000057
Train Epoch: 33 [10/79 (13%)]	Train Loss: 0.000751
Train Epoch: 33 [20/79 (25%)]	Train Loss: 0.002666
Train Epoch: 33 [30/79 (38%)]	Train Loss: 0.005316
Train Epoch: 33 [40/79 (51%)]	Train Loss: 0.000024
Train Epoch: 33 [50/79 (63%)]	Train Loss: 0.001403
Train Epoch: 33 [60/79 (76%)]	Train Loss: 0.000015
Train Epoch: 33 [70/79 (89%)]	Train Loss: 0.000701

Train set: Average loss: 0.0000, Accuracy: 9919/10000 (99%)


Test set: Average loss: 0.0001, Accuracy: 9947/10000 (99%)

Train Epoch: 34 [0/79 (0%)]	Train Loss: 0

Train Epoch: 47 [70/79 (89%)]	Train Loss: 0.000537

Train set: Average loss: 0.0000, Accuracy: 9940/10000 (99%)


Test set: Average loss: 0.0002, Accuracy: 9941/10000 (99%)

Train Epoch: 48 [0/79 (0%)]	Train Loss: 0.002405
Train Epoch: 48 [10/79 (13%)]	Train Loss: 0.000206
Train Epoch: 48 [20/79 (25%)]	Train Loss: 0.001759
Train Epoch: 48 [30/79 (38%)]	Train Loss: 0.000213
Train Epoch: 48 [40/79 (51%)]	Train Loss: 0.000827
Train Epoch: 48 [50/79 (63%)]	Train Loss: 0.001363
Train Epoch: 48 [60/79 (76%)]	Train Loss: 0.000176
Train Epoch: 48 [70/79 (89%)]	Train Loss: 0.000697

Train set: Average loss: 0.0000, Accuracy: 9949/10000 (99%)


Test set: Average loss: 0.0002, Accuracy: 9946/10000 (99%)

Train Epoch: 49 [0/79 (0%)]	Train Loss: 0.000091
Train Epoch: 49 [10/79 (13%)]	Train Loss: 0.001273
Train Epoch: 49 [20/79 (25%)]	Train Loss: 0.000027
Train Epoch: 49 [30/79 (38%)]	Train Loss: 0.001890
Train Epoch: 49 [40/79 (51%)]	Train Loss: 0.000035
Train Epoch: 49 [50/79 (63%)]	Train Loss: 0.

In [59]:
torch.save(model.state_dict(), "mnist.pt")